# Model Evaluation for L2RPN

In [45]:
import grid2op
from grid2op.Agent.doNothing import DoNothingAgent
from grid2op.Runner import Runner
from grid2op.Episode import EpisodeData
from grid2op.Reward import RedispReward, IncreasingFlatReward, AlarmReward, CombinedScaledReward
from grid2op.utils import EpisodeStatistics, ScoreL2RPN2020, ScoreICAPS2021, ScoreL2RPN2022
from pathlib import Path
from typing import List, Dict
import pandas as pd

from tqdm.notebook import tqdm

## Settings

In [39]:
env_name: str = "rte_case14_realistic"
evaluation_folder_path: str = "/home/l2rpn/data/evaluation"
validation_suffix: str = "_val10"
difficulty = "competition"
number_of_episodes = 100



### Environment Initialization

In [40]:
# Initialize Environment
env = grid2op.make(env_name + validation_suffix, reward_class=CombinedScaledReward)

# Initialized Combined Reward similar to L2RPN Score
combined_reward: CombinedScaledReward = env.get_reward_instance() 
combined_reward.addReward("IncreasingFlat", IncreasingFlatReward(), 0.7)
combined_reward.addReward("Redispatching", RedispReward(), 0.7)
if env.alarms_area_lines:
    combined_reward.addReward("Alarm", AlarmReward(), 0.3)
combined_reward.initialize(env)

# Initialize Runner Parameters
runner_parameters = env.get_params_for_runner()
runner_parameters["verbose"] = True


### Agents

#### Do nothing

In [42]:
agent = DoNothingAgent(env.action_space)

## Generate Do Nothing Data

In [ ]:
do_nothing_evaluation_folder: Path = Path(evaluation_folder_path, "do_nothing_" + env_name)
    
if not do_nothing_evaluation_folder.exists():
    do_nothing_evaluation_folder.mkdir(parents=True, exist_ok=False)
    runner = Runner(**runner_parameters, agentClass=DoNothingAgent)
    runner.run(nb_episode=number_of_episodes, path_save=str(do_nothing_evaluation_folder), pbar=tqdm)

## Retrieving Data Generated by the Runner on Validation Set

In [ ]:
agent_to_episodes: Dict[str, List[int]] = (
{agent_folder_path.name: 
 [ EpisodeData.from_disk(agent_folder_path, episode_folder_path.name)
  for episode_folder_path in tqdm(list(filter(lambda x: x.is_dir() and str(x.name).isdigit(), 
                        Path(evaluation_folder_path, agent_folder_path).iterdir())))] 
 for agent_folder_path in tqdm(list(filter(lambda x: x.is_dir(), 
                          Path(evaluation_folder_path).iterdir())))})

## Score

In [ ]:
# Score Settings
score_l2rpn2022 = ScoreL2RPN2022(env, nb_scenario=number_of_episodes)        

In [ ]:
# Agent Score
agent_score = score_l2rpn2022.get(agent)

In [ ]:
# Save Score to file
agent_score_df = pd.DataFrame(agent_score).transpose()
agent_score_df.columns = ["all_scores", "ts_survived", "total_ts"]


In [58]:
agent_score_df.to_csv(Path(do_nothing_evaluation_folder, "l2rpn_2022_score.csv"))